In [ ]:
from skimage.io import imsave
import numpy as np


# Define input path

In [ ]:
# path to the folder containing the images of a certain channel

filepath = '/media/eric/WD_Elements/EPFL-data/Lucas/'


Z_slices = 19
xdim = 1200
ydim = 1200

# input can be a 3D time series, but only one channel (T, Z, X, Y)
fn_channel = 'green_20230920_AirGel_WTmSc_dwspFmN_mix_LAM02J-L_CIP_5.5hpi_23.5h.nd...el_WTmSc_dwspFmN_mix_LAM02J-L_CIP_5.5hpi_23.5h.nd2 (series 08) - C=0.tif'

channel_name = "green" # rename it to WT or wsp or whatever mutant




full_fn = filepath + fn_channel

# if orange endswith .nd2 then import pims_nd2
if full_fn.endswith('.nd2'):
    !pip install pims_nd2
    import pims_nd2 as pims

else:
    !pip install pims
    import pims


In [ ]:
import napari
%gui qt

def Draw_and_Save(img,output_dir,output_fn):
    viewer = napari.Viewer()
    viewer.add_image(img)
    labels = viewer.add_labels(np.zeros(img.shape,dtype='uint8'))

    print("before block")
    viewer.show(block=True)

    print("after block")
    # get the drawings first
    manual_annotations = labels.data

    # save the drawings
    manual_labels_filename = output_dir + '/' + output_fn
    imsave(manual_labels_filename, manual_annotations, check_contrast=False)
    viewer.close()
    del viewer, labels, manual_annotations


# Prepare GT and save it together with the image

In [ ]:
data_dir = '../data_3D/'

In [ ]:
# create output directory
import os
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

if not os.path.exists(data_dir + channel_name):
    os.makedirs(data_dir + channel_name + "GT")
    os.makedirs(data_dir + channel_name + "RAW")


In [ ]:
stack_seq = pims.open(fn_channel)

timestep = 0
counter = 0
for t, img_slice in enumerate(stack_seq):
    

    # iterate through all z slices and stack them together
    if np.mod(t,Z_slices) == 0:
        # prepare empty array with the same dtype as the first image
        counter = 0
        img_stack = np.zeros((Z_slices,xdim,ydim))

    img_stack[counter,:,:] = img_slice

    if np.mod(t,Z_slices) == Z_slices-1:
        print("Timestep: ", timestep)

        # now there is a 3D image stack of 19 slices
        # use napari to draw the segmentation, then save it
        Draw_and_Save(img_stack, data_dir + channel_name + "GT/", 'T_'+str(timestep).zfill(3)+ '_' + fn_channel)
        imsave(data_dir + channel_name + "RAW/" + 'T_'+str(timestep).zfill(3)+  '_' + fn_channel , img_stack,check_contrast=False)

        del img_stack

        timestep += 1

    counter += 1

